In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_x = train.drop(["Survived"],axis=1)
train_y = train["Survived"]

In [ ]:
test_x = test.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
train_x = train_x.drop(["PassengerId"], axis=1)
test_x = test_x.drop(["PassengerId"], axis=1)
train_x.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_x = train_x.drop(["Name", "Ticket", "Cabin"], axis=1)
test_x = test_x.drop(["Name", "Ticket", "Cabin"], axis=1)

In [ ]:
for c in ["Sex", "Embarked"]:
  le = LabelEncoder()
  le.fit(train_x[c].fillna("NA"))
  train_x[c] = le.transform(train_x[c].fillna("NA"))
train_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,3
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,3
3,1,0,35.0,1,0,53.1000,3
4,3,1,35.0,0,0,8.0500,3


In [ ]:
for c in ["Sex", "Embarked"]:
  le = LabelEncoder()
  le.fit(test_x[c].fillna("NA"))
  test_x[c] = le.transform(test_x[c].fillna("NA"))
test_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [ ]:
from xgboost import  XGBClassifier

model = XGBClassifier(n_estimators = 20, random_state= 71)
model.fit(train_x, train_y)

pred = model.predict_proba(test_x)[:,1]

pred_label = np.where(pred > 0.5,1,0)

submission = pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":pred_label})
submission.to_csv("submission_first.csv", index=False)

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

scores_accuracy = []
scores_logloss = []

kf = KFold(n_splits = 4, shuffle=True, random_state = 71)
for tr_idx, va_idx in kf.split(train_x):
  tr_x,va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
  tr_y,va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

  model = XGBClassifier(n_estimators=20, random_stat=71)
  model.fit(tr_x,tr_y)

ccccc


In [ ]:
import itertools

param_space = {
    "max_depth":[3,5,7],
    "min_child_weight":[1.0,2.0,4.0]
}

param_combinations = itertools.product(param_space["max_depth"],param_space["min_child_weight"])

params =[]
scores = []

for max_depth, min_child_weight in param_combinations:
  score_folds = []
  kf = KFold(n_splits=4, shuffle=True,random_state=123456)
  for tr_idx, va_idx in kf.split(train_x):
    tr_x,tr_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y,tr_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    model = XGBClassifier(n_estimators=20, random_state=71, max_depth=max_depth, min_child_weight=min_child_weight)
    model.fit(tr_x,tr_y)

    va_pred = model.predict_proba(va_x)[:,1]
    logloss = log_loss(va_y,va_pred)
    score_folds.append(logloss)

  score_mean = np.mean(score_folds)

  params.append((max_depth, min_child_weight))
  scores.append(score_mean)

best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth:{best_param[0]}, min_child_weight:{best_param[1]}')

max_depth:7, min_child_weight:1.0


In [ ]:
logloss:0.6869,accuracy:0.571864

In [ ]:
import copy
train_x2 = copy.deepcopy(train_x)
test_x2 = copy.deepcopy(test_x)
train_y2 = copy.deepcopy(train_y)

test_x2["Fare"] = test_x2["Fare"].replace(0,10.5)
test_x2["Fare"] = test_x2["Fare"].fillna(10.5) 

age_avg = test_x2['Age'].mean()
age_std = test_x2['Age'].std()
test_x2['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)


age_avg = train_x2['Age'].mean()
age_std = train_x2['Age'].std()
train_x2['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)

test_x2["Fare"] = test_x2["Fare"].replace(0,10.5)

In [ ]:
from sklearn.linear_model import LogisticRegression

model_xgb = XGBClassifier(n_estimators=20,random_state=71)
model_xgb.fit(train_x,train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:,1]

model_lr = LogisticRegression(solver="lbfgs", max_iter=300)
model_lr.fit(train_x2, train_y2)
pred_lr = model_lr.predict_proba(test_x2)[:,1]

pred = pred_xgb*0.8 + pred_lr*0.2
pred_label = np.where(pred>0.5,1,0)

submission = pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":pred_label})
submission.to_csv("submission.csv", index=False)

In [ ]:
from xgboost import  XGBClassifier

model = XGBClassifier(n_estimators = 20, random_state= 71)
model.fit(train_x, train_y)

pred = model.predict_proba(test_x)[:,1]

pred_label = np.where(pred > 0.5,1,0)

submission = pd.DataFrame({"id":test["id"].astype('int64'),"y":pred_label})
submission.to_csv("submission_first.csv", index=False)

In [ ]:
from xgboost import  XGBClassifier
model1 = XGBClassifier(n_estimators = 20, random_state= 71)
model2 = XGBClassifier(n_estimators = 20, random_state= 71)
if train_x[train_x["lobby-mode"]==1]: #1の時はnawabari
    model1.fit(train_x["stage","A1-weapon", "A2-weapon", "A3-weapon","A4-weapon","B1-weapon","B2-weapon","B3-weapon","B4-weapon","A1-level", "A2-level", "A3-level","A4-level","B1-level","B2-level","B3-level","B4-level","A_member","B_member"], train_y)
    pred = model1.predict_proba(test_x)[:,1]
    pred_label = np.where(pred > 0.5,1,0)
if train_x[train_x["lobby-mode"]==0]: #0の時はgachi
    model1.fit(train_x["stage","A1-weapon", "A2-weapon", "A3-weapon","A4-weapon","B1-weapon","B2-weapon","B3-weapon","B4-weapon","A1-level", "A2-level", "A3-level","A4-level","B1-level","B2-level","B3-level","B4-level","A_member","B_member""A1-rank", "A2-rank", "A3-rank","A4-rank","B1-rank","B2-rank","B3-rank","B4-rank","lobby","mode"], train_y)
    pred = model2.predict_proba(test_x)[:,1]
    pred_label = np.where(pred > 0.5,1,0)
submission = pd.DataFrame({"id":test["id"].astype('int64'),"y":pred_label})
submission.to_csv("submission_first.csv", index=False)